Import Panda und SQLalchemy

In [4]:
import pandas as pd
import sqlalchemy as sql
import numpy as np

csv-File importieren.

In [16]:
bln_22_23 = pd.read_csv("data/Berlin_22-23.csv", delimiter= ",", encoding= 'utf-8')

In [17]:
bln_22_23.head(10)

,ID,Typ,Bezeichnung,Bereich,Bereichsbezeichnung,Einzelplan,Einzelplanbezeichnung,Kapitel,Kapitelbezeichnung,Hauptgruppe,...,Oberfunktion,Oberfunktionsbezeichnung,Funktion,Funktionsbezeichnung,Titelart,Titel,Titelbezeichnung,Jahr,BetragTyp,Betrag
0,1,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,11201,"Geldstrafen, Geldbußen, Verwarnungs- und Zwang...",2022,Soll,1000
1,2,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,11906,Ersatz von Fernmeldegebühren,2022,Soll,1000
2,3,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,11961,Erstattung von Steuerbeträgen,2022,Soll,1000
3,4,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,11979,Verschiedene Einnahmen,2022,Soll,10000
4,5,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,12401,"Mieten für Grundstücke, Gebäude und Räume",2022,Soll,35000
5,6,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,12406,Mieten für Stellplätze auf Dienstgrundstücken,2022,Soll,28800
6,7,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,2,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,23601,Ersatz von Ausgaben durch Sozialversicherungst...,2022,Soll,1000
7,8,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,4,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Ausgabetitel,41101,Aufwendungen für Abgeordnete,2022,Soll,38559000
8,9,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,4,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Ausgabetitel,41103,Versorgungsbezüge der ehemaligen Abgeordneten,2022,Soll,8417000
9,10,2,Verfassungsorgane,30,Hauptverwaltung,1,Abgeordnetenhaus,100,Abgeordnetenhaus,4,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Ausgabetitel,42201,Bezüge der planmäßigen Beamtinnen und Beamten,2022,Soll,5099000


Drop die ersten vier Columns, die nicht gebraucht werden

In [18]:
bln_22_23 = bln_22_23.drop(['ID','Typ', 'Bezeichnung', 'Bereich', 'Bereichsbezeichnung'], axis=1)

Columns names anpassen, Sonderzeichen entfernen, in tausend Euro entfernen

In [19]:
bln_22_23.columns = bln_22_23.columns.str.replace(" ", "_")
bln_22_23.columns = bln_22_23.columns.str.lower()
bln_22_23


,einzelplan,einzelplanbezeichnung,kapitel,kapitelbezeichnung,hauptgruppe,hauptgruppenbezeichnung,obergruppe,obergruppenbezeichnung,gruppe,gruppenbezeichnung,...,oberfunktion,oberfunktionsbezeichnung,funktion,funktionsbezeichnung,titelart,titel,titelbezeichnung,jahr,betragtyp,betrag
0,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,"Verwaltungseinnahmen, Einnahmen aus Schuldendi...",11,Verwaltungseinnahmen,112,"Geldstrafen, Geldbußen und Zwangsgelder (einsc...",...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,11201,"Geldstrafen, Geldbußen, Verwarnungs- und Zwang...",2022,Soll,1000
1,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,"Verwaltungseinnahmen, Einnahmen aus Schuldendi...",11,Verwaltungseinnahmen,119,Sonstige Verwaltungseinnahmen,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,11906,Ersatz von Fernmeldegebühren,2022,Soll,1000
2,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,"Verwaltungseinnahmen, Einnahmen aus Schuldendi...",11,Verwaltungseinnahmen,119,Sonstige Verwaltungseinnahmen,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,11961,Erstattung von Steuerbeträgen,2022,Soll,1000
3,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,"Verwaltungseinnahmen, Einnahmen aus Schuldendi...",11,Verwaltungseinnahmen,119,Sonstige Verwaltungseinnahmen,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,11979,Verschiedene Einnahmen,2022,Soll,10000
4,1,Abgeordnetenhaus,100,Abgeordnetenhaus,1,"Verwaltungseinnahmen, Einnahmen aus Schuldendi...",12,Einnahmen aus wirtschaftlicher Tätigkeit und a...,124,Mieten und Pachten,...,1,Politische Führung und zentrale Verwaltung,11,Politische Führung,Einnahmetitel,12401,"Mieten für Grundstücke, Gebäude und Räume",2022,Soll,35000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46830,45,Allgemeine Finanzangelegenheiten,4520,Stiftungen (ohne Heime),5,"Sächliche Verwaltungsausgaben, militärische Be...",51,Sächliche Verwaltungsausgaben,517,"Bewirtschaftung der Grundstücke, Gebäude und R...",...,81,"Grund- und Kapitalvermögen, Sondervermögen",813,Sondervermögen,Ausgabetitel,51790,"Bewirtschaftung der Grundstücke, Gebäude und R...",2023,Soll,2600
46831,45,Allgemeine Finanzangelegenheiten,4520,Stiftungen (ohne Heime),5,"Sächliche Verwaltungsausgaben, militärische Be...",51,Sächliche Verwaltungsausgaben,519,Unterhaltung der Grundstücke und baulichen Anl...,...,81,"Grund- und Kapitalvermögen, Sondervermögen",813,Sondervermögen,Ausgabetitel,51990,Unterhaltung der Grundstücke und baulichen Anl...,2023,Soll,6500
46832,45,Allgemeine Finanzangelegenheiten,4520,Stiftungen (ohne Heime),8,Sonstige Ausgaben für Investitionen und Invest...,83,Erwerb von Beteiligungen und dgl.,831,Erwerb von Beteiligungen und dgl. im Inland,...,81,"Grund- und Kapitalvermögen, Sondervermögen",812,Kapitalvermögen,Ausgabetitel,83190,Kauf von Wertpapieren aus zweckgebundenen Einn...,2023,Soll,1000
46833,45,Allgemeine Finanzangelegenheiten,4520,Stiftungen (ohne Heime),9,Besondere Finanzierungsausgaben,91,"Zuführungen an Rücklagen, Fonds und Stöcke",919,Zuführungen an sonstige Rücklagen,...,85,Rücklagen,850,Rücklagen,Ausgabetitel,91990,Zweckgebundene Zuführung an den Geldbestand,2023,Soll,1000


Values ins passende Format setzten und leading-0 einfügen, wo sie automatisch entfernt wurde

In [ ]:
bln_22_23['einzelplan'] = bln_22_23['einzelplan'].astype(str)
bln_22_23['kapitel'] = bln_22_23['kapitel'].astype(str)

bln_22_23['hauptgruppe'] = bln_22_23['hauptgruppe'].astype(str)
bln_22_23['kapitel'] = bln_22_23['kapitel'].astype(str)
bln_22_23['kapitel'] = bln_22_23['kapitel'].astype(str)

bln_22_23['titel'] = bln_22_23['titel'].astype(str)
bln_22_23['titel'] = bln_22_23['titel'].apply(lambda x:x.zfill(5))
bln_22_23['ep'] = bln_22_23['ep'].apply(lambda x:x.zfill(2))
bln_22_23['kapitel'] = bln_22_23['kapitel'].str[-2:]
bln_22_23

In [ ]:
def bln_22_23

Titel in gruppe und counter gesplittet, gruppe und counter eingefügt

In [ ]:
bln_22_23["gruppe"] = bln_22_23["titel"].str[:3]
bln_22_23["counter"] = bln_22_23["titel"].str[3:]
bln_22_23.info()

jahr und state_id eingefügt

In [ ]:
bln_22_23.insert(8, "jahr", 2022)
bln_22_23.insert(9, "state_id", 15)
bln_22_23.info()

titel gedropt, da redundant

In [ ]:
bln_22_23 = bln_22_23.drop("titel", axis=1)
bln_22_23.head(20)

anhand der gruppe einnahmen und ausgaben in den Zahlen definiert

In [ ]:
bln_22_23['ansatz_2022_euro'] =bln_22_23[['ansatz_2022_euro', 'gruppe']].apply(lambda x:-x[0] if int(x[1]) > 400 else x[0], axis=1)


mgtg gedropt, hat keinen mehrwert und viele nan

In [ ]:
bln_22_23 = bln_22_23.drop(['mgtg'], axis=1)


einmal angucken, da schön

In [ ]:
bln_22_23.head()